# Script to convert OIDv6 labels to YOLO Format

In [2]:
import os
import shutil
from PIL import Image
from tqdm import tqdm

# Paths
imgs_dir    = 'OID_Person/OIDv6/test/person'
labels_dir  = 'OID_Person/OIDv6/test/person/labels'
out_dir     = 'coco2017_person/test'

# Make sure output dir exists
os.makedirs(out_dir, exist_ok=True)

# Map your class names to YOLO indices
class_map = {'person': 0}

# Gather all label files
label_files = [f for f in os.listdir(labels_dir) if f.endswith('.txt')]

for fname in tqdm(label_files, desc='Converting to YOLO', unit='file'):
    img_id    = os.path.splitext(fname)[0]
    img_src   = os.path.join(imgs_dir,   img_id + '.jpg')
    label_in  = os.path.join(labels_dir, fname)
    label_out = os.path.join(out_dir,     fname)

    # Load image size
    with Image.open(img_src) as img:
        W, H = img.size

    lines_out = []
    with open(label_in) as f:
        for line in f:
            cls, xmin, ymin, xmax, ymax = line.split()
            xmin, ymin, xmax, ymax = map(float, (xmin, ymin, xmax, ymax))

            # Compute centre-based, normalised coords
            cx = (xmin + xmax) / 2.0
            cy = (ymin + ymax) / 2.0
            w  = xmax - xmin
            h  = ymax - ymin

            x_norm = cx / W
            y_norm = cy / H
            w_norm = w  / W
            h_norm = h  / H

            idx = class_map[cls]
            lines_out.append(f"{idx} {x_norm:.6f} {y_norm:.6f} {w_norm:.6f} {h_norm:.6f}")

    # Write YOLO-style label
    with open(label_out, 'w') as f:
        f.write("\n".join(lines_out))

    # Copy image alongside its label in the same directory
    img_dst = os.path.join(out_dir, img_id + '.jpg')
    shutil.copyfile(img_src, img_dst)


Converting to YOLO: 100%|██████████| 21688/21688 [04:10<00:00, 86.60file/s] 
